In [1]:
import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization, Input
from keras.layers import Conv2D, MaxPool2D
from keras.models import Model
from keras.datasets import cifar10
from keras import regularizers
from keras.callbacks import LearningRateScheduler
import numpy as np
import weave
import json
import time

/Users/Tim/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
keras.backend.image_data_format()

'channels_first'

In [3]:
def lr_schedule(epoch):
    return (0.00100 - 7e-6*epoch)

def straight_stack(x, num_filters, rep = 3, weight_decay = 1e-4, drop = 0.2):
    for n in range(rep):
        x = Conv2D((2**n)*num_filters, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay))(x)
        x = BatchNormalization()(x)
        x = Conv2D((2**n)*num_filters, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay))(x)
        x = BatchNormalization()(x)
        x = MaxPool2D()(x)
        x = Dropout(drop)(x)
    
    return x

class TimeHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

In [4]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

print(x_train.shape)
 
#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)
 
num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)

datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )
datagen.fit(x_train)

batch_size = 64

(50000, 3, 32, 32)


In [5]:
# viewer = keras.callbacks.TensorBoard(log_dir='./logs', 
#                                      histogram_freq=0, 
#                                      batch_size=batch_size, 
#                                      write_graph=True)

conv_layers = 3
filters = [4,8,16,32]
num_val_test = 500
num_final_test = 10000
epochs = 1
steps_per_epoch = 1 #x_train.shape[0] // batch_size
model_details = {}
drop = 0.2
for num_filters in filters:
    for num_stack in [4,2,1]:
        time_callback = TimeHistory()
        inputs = Input(shape=(3,32,32))
        input_array = []
        pyrm_layers = int(np.log2(num_stack))
        for stack in range(num_stack):
            x = straight_stack(inputs, num_filters/num_stack, rep = conv_layers - pyrm_layers, drop = drop)
            input_array.append(x)
        if pyrm_layers > 0:
            input_size = tuple(input_array[0].shape[1:].as_list())
            x = weave.pyrm_net(inputs = input_array,
                               n_layers = pyrm_layers,
                               n_filters_start = input_size[0]*2,
                               n_gpus = 1,
                               r_filter = 4,
                               r_combine= 2,
                               min_dim = 2,
                               max_pool_loc = 1,
                               pure_combine = False)
            x = BatchNormalization()(x)
            x = Conv2D(x.shape.as_list()[1], (3,3), padding='same', activation='relu')(x)
            x = BatchNormalization()(x)
            x = MaxPool2D()(x)
            x = Dropout(drop)(x)
        else:
            pass
        x = Flatten()(x)

        predictions = Dense(num_classes, activation='softmax')(x)
        model = Model(inputs=[inputs], outputs=predictions)
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        num_params = model.count_params()
        print('number of filters %d, number of stack %d' % (num_filters, num_stack))
        print('number of parameters %d' % num_params)
        hist = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                                    steps_per_epoch= steps_per_epoch,
                                    epochs=epochs,
                                    verbose=0,
                                    validation_data=(x_test[0:num_val_test],y_test[0:num_val_test]),
                                    callbacks=[LearningRateScheduler(lr_schedule), time_callback]).history
        hist['num_params'] = num_params
        hist['num_filters'] = num_filters
        hist['final accuracy'] = model.evaluate(x_test[0:num_final_test],y_test[0:num_final_test], verbose= 0)
        hist['model'] = model.to_json()
        hist['times'] = time_callback.times
        filepath = str(num_filters) + '_' + str(num_stack) + '_' + str(epochs)
        with open(filepath+'.json', 'w') as outfile:
            json.dump(hist, outfile)
        model.save(filepath+'.h5')
    
    
    

number of filters 4, number of stack 4
number of parameters 8202
number of filters 4, number of stack 2
number of parameters 8150
number of filters 4, number of stack 1
number of parameters 7646
number of filters 8, number of stack 4
number of parameters 23874
number of filters 8, number of stack 2
number of parameters 24530
number of filters 8, number of stack 1
number of parameters 23762
number of filters 16, number of stack 4
number of parameters 80922
number of filters 16, number of stack 2
number of parameters 84506
number of filters 16, number of stack 1
number of parameters 82778
number of filters 32, number of stack 4
number of parameters 297834
number of filters 32, number of stack 2
number of parameters 313322
number of filters 32, number of stack 1
number of parameters 307946
